In [1]:
%matplotlib inline
import glob
import random
import math
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches
import seaborn as sns

import skimage
from skimage.external import tifffile
from scipy import stats
from skimage.measure import label, regionprops, compare_ssim, compare_nrmse, compare_psnr
from skimage.morphology import remove_small_objects, skeletonize, binary_opening, disk
from skimage.segmentation import clear_border
from skimage.filters import threshold_li
from skimage.transform import rotate

matplotlib.rcParams['figure.figsize'] = (20.0, 15.0)

# Import Labels

In [2]:
daf7 = pd.read_csv('raw_data/labels/0_daf-7.dat')
gld1 = pd.read_csv('raw_data/labels/1_gld-1_RNAi.dat')
glp1 = pd.read_csv('raw_data/labels/2_glp-1_gf.dat')
hm_0 = pd.read_csv('raw_data/labels/3_HM_08_03.dat')
hm_1 = pd.read_csv('raw_data/labels/4_HM_08_15.dat')
hm_2 = pd.read_csv('raw_data/labels/5_HM_08_23.dat')
hm_3 = pd.read_csv('raw_data/labels/6_HM_08_25.dat')

all_labels = pd.concat([
    daf7['Definite Movement Calls 410/470'],
    gld1['Def Movement Call 410/470'],
    glp1['Movement 470/470'],
    hm_0['Movement 470/470'],
    hm_1['Movement 470/470'],
    hm_2['Movement 470/470'],
    hm_3['Movement 470/470']],
    ignore_index=True,
)
problem_labels = np.where(np.isnan(all_labels))[0]
all_labels = np.asarray(all_labels, dtype=int)

# Import Images

In [ ]:
def get_imset_num(path):
    return path.split('/')[2].split('_')[0]

def get_wavelength_imset_paths(wavelength):
    return sorted([glob.glob('raw_data/imgs/%s*_%d_1*' % (i, wavelength))[0] for i in set([get_imset_num(s) for s in glob.glob('raw_data/imgs/*')])],
                  key=lambda path: get_imset_num(path))

_410_img_paths = get_wavelength_imset_paths(410)
_470_img_paths = get_wavelength_imset_paths(470)

all_410 = np.concatenate([tifffile.imread(_410_img_paths[i]) for i in range(len(_410_img_paths))], axis=0)
all_470 = np.concatenate([tifffile.imread(_470_img_paths[i]) for i in range(len(_470_img_paths))], axis=0)

problem_ims = [1156]
problem_idx = np.append(problem_labels, problem_ims)

all_410 = np.delete(all_410, problem_idx, axis=0)
all_470 = np.delete(all_470, problem_idx, axis=0)
all_labels = np.delete(all_labels, problem_idx)

# Thresholding

In [ ]:
def make_bin_mask(img):
    thresh = img>threshold_li(img)
    try:
        return remove_all_but_rightmost(remove_small_objects(thresh))
    except:
        return remove_all_but_rightmost(thresh)
    
